# Simple reproduction - GEO-AI Challenge for Cropland Mapping by ITU
_Antoine Saget_

In this notebook, the results are reproduced with a more streamlined code that might be easier to integrate in a production pipeline than the original code as the original code was written only for the purpose of the challenge.

In [9]:
##### PLEASE SE THIS VARIABLE TO YOUR PROJECT NAME IN GEE #####
PROJECT_NAME = 'ee-antoinesaget'

In [10]:
# Imports and seeds initializations
import random
import ee

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier

# Set seeds for reproducibility
SEED = 2023
random.seed(SEED)
np.random.seed(SEED)

In [11]:
# ee.Authenticate()
ee.Initialize()

In [12]:
# Constants

# Dataframe column names
TS_ID = 'TS_ID'
ID = 'ID'
TARGET = 'Target'
LAT = 'Lat'
LON = 'Lon'
TIMESTAMP = 'Timestamp'
COUNTRY = 'Country'
IS_TRAIN = 'IsTrain'

B2, B3, B4, B5, B6, B7, B8, B8A, B11, B12 = 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'
SCL = 'SCL'
NDVI = 'NDVI'

ALL_BANDS = [B2, B3, B4, B5, B6, B7, B8, B8A, B11, B12, SCL, NDVI]
BEST_BANDS = [B3, B4, B8, LON, LAT, NDVI]

SUDAN = 'Sudan'
AFGHANISTAN = 'Afghanistan'
IRAN = 'Iran'

COUNTRY_NAME = 'Country Name'
START_DATE = 'Start Date'
END_DATE = 'End Date'
BOUNDS = 'Bounds'

# GEE Collection name
COLLECTION_NAME = 'COPERNICUS/S2_SR_HARMONIZED'

CACHE_FOLDER = 'data/'

# Country bounds and timeranges
country_settings = {
   SUDAN: {
        COUNTRY_NAME: SUDAN,
        START_DATE: '2019-07-01',
        END_DATE: '2020-06-30',
        BOUNDS: [[14.1, 33.1], [14.6, 33.6]]
    },
    AFGHANISTAN: {
        COUNTRY_NAME: AFGHANISTAN,
        START_DATE: '2022-04-01',
        END_DATE: '2022-04-30',
        BOUNDS: [[34.0, 70.2], [34.4, 70.8]]
    },
    IRAN: {
        COUNTRY_NAME: IRAN,
        START_DATE: '2019-07-01',
        END_DATE: '2020-06-30',
        BOUNDS: [[32.0, 48.1], [32.5, 48.6]]
    }
}

BASENAME = f'projects/{PROJECT_NAME}/assets/'

In [24]:
# Utils functions
def filter_by_bounds_mask(points, bounds):
    """
    Returns a boolean mask of the points that are inside the bounds
    """
    min_, max_ = bounds
    min_lat, min_lon = min_
    max_lat, max_lon = max_

    return (
        (points[LAT] >= min_lat) &
        (points[LAT] <= max_lat) &
        (points[LON] >= min_lon) &
        (points[LON] <= max_lon)
    )

def filter_by_country(df, country):
    if country is None:
        return np.ones(df.shape[0]).astype('bool')
    return df[COUNTRY] == country[COUNTRY_NAME]

def filter_by_dates(points, start_date, end_date):
    return points.loc[
        (points[TIMESTAMP] >= start_date) &
        (points[TIMESTAMP] <= end_date)
    ]

def fc_to_dict(fc):
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
        reducer=ee.Reducer.toList().repeat(prop_names.size()),
        selectors=prop_names).get('list')

    return ee.Dictionary.fromLists(prop_names, prop_lists)

def prepare_df(df, stats_dict):
    bands = ALL_BANDS
    stats_df = pd.DataFrame(stats_dict)

    def add_date_info(df):
        df[TIMESTAMP] = pd.to_datetime(df['millis'], unit='ms')
        return df

    stats_df = add_date_info(stats_df)
    stats_df = stats_df.drop(columns=['millis', 'system:index'])
    
    stats_df['TS_ID'] = stats_df['TS_ID'].astype('uint')
    ts_ids = np.unique(stats_df['TS_ID'])

    assert len(ts_ids) == len(df)

    for ts_id, id in zip(np.unique(stats_df['TS_ID']), df.index.values):
        stats_df.loc[stats_df['TS_ID'] == ts_id, ID] = id
    stats_df = stats_df.drop(columns=['TS_ID'])
        
    stats_df.sort_values(by=[ID, TIMESTAMP], inplace=True)

    return stats_df

def start_download_task(df, country, start_date, end_date, bands, filename, scale=10):
    datapoints = df[filter_by_country(df, country)]
    points_fc = fc_from_points(datapoints)
    collection = (ee
        .ImageCollection(self.collection_name)
        .filterDate(start_date, end_date)
        .filterBounds(points_fc)
    )

    if 'NDVI' in bands:
        collection = collection.map(addNDVI)

    def image_reducer(image):
        def feature_reducer(feature):
            coordinates = feature.geometry().coordinates()
            di = {
                'millis': image.date().millis(),
                TS_ID: feature.id(),
                LON: coordinates.get(0),
                LAT: coordinates.get(1)
            }
            
            for band in bands:
                band_filtered = ee.List([feature.get(band), -9999]).reduce(ee.Reducer.firstNonNull())
                di[band] = band_filtered

            return feature.set(di)

        return image.select(bands).reduceRegions(
            collection=points_fc,
            reducer=ee.Reducer.first(),
            scale=scale,
        ).map(feature_reducer)

    stats_fc = (ee.FeatureCollection(collection
        .map(image_reducer)
        .flatten()
        .filter(ee.Filter.neq(bands[0], -9999))
        .distinct(['system:index', 'millis'])
    ))

    task = ee.batch.Export.table.toAsset(
        collection=stats_fc,
        description='stats_fc export',
        assetId=self.basename + filename,
    )
    task.start()
    return task

def interpolate_nans_3D(X):
    n_objects, n_timesteps, n_bands = X.shape
    isnan = np.isnan(X[:, :, 0])
    for i, x in enumerate(X):
        valid_timesteps = np.arange(n_timesteps)[~isnan[i]]
        valid_values = X[i, valid_timesteps]

        # Perform linear interpolation and extrapolation
        for j in range(n_bands):
            X[i, :, j] = np.interp(
                np.arange(n_timesteps), 
                valid_timesteps,
                valid_values[:, j]
            )
    return X

def interpolate_ts(df, bands, sampling_rate=5, start_date=None, end_date=None):
    df = df.sort_values(by=[ID, TIMESTAMP])

    # bands_train is a list of timeseries of shape (n_timesteps, n_bands)
    # but each timeserie can have a different number of timestamps so we will interpolate 
    # the missing values between start and end date

    if start_date is None:
        start_date = pd.to_datetime(df[TIMESTAMP].min())
    if end_date is None:
        end_date = pd.to_datetime(df[TIMESTAMP].max())
    date_range = pd.date_range(start_date, end_date, freq='D', normalize=True)

    df[TIMESTAMP] = pd.to_datetime(df[TIMESTAMP]).dt.normalize()
    df = df.drop_duplicates(subset=[ID, TIMESTAMP])

    groups = df.groupby(ID)
    XX = groups.apply(
        lambda x: x.set_index(TIMESTAMP)[bands].reindex(date_range).values
    )
    IDs = groups[ID].first().values

    XX = np.stack(XX.values).astype('float32')
    XX = interpolate_nans_3D(XX)
    XX = XX[:, ::sampling_rate, :]

    return XX, IDs

def save_submission(preds, ids, filename):
    df = pd.DataFrame({'ID': ids, 'Target': preds})
    time = pd.Timestamp.now().strftime('%m_%d_%Hh_%Mm_%Ss')
    folder = 'submissions'
    
    filename = f'{folder}/{time}_{filename}.csv'
    
    df.to_csv(filename, index = False)

## 1. Data loading

In [25]:
# Load the dataset
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

# A single pandas Dataframe for test and train with Target at null for test
df = pd.concat([train, test])

# Add a convenient column to distinguish between train and test
df[IS_TRAIN] = df[TARGET].notnull()

# Add a column with the country name of each point
for country in country_settings.values():
    mask = filter_by_bounds_mask(df, country[BOUNDS])
    df.loc[mask, COUNTRY] = country[COUNTRY_NAME]

df = df.set_index(ID)

display(df)

,Lat,Lon,Target,IsTrain,Country
ID,,,,,
ID_SJ098E7S2SY9,34.162491,70.763668,0.0,True,Afghanistan
ID_CWCD60FGJJYY,32.075695,48.492047,0.0,True,Iran
ID_R1XF70RMVGL3,14.542826,33.313483,1.0,True,Sudan
ID_0ZBIDY0PEBVO,14.359480,33.284108,1.0,True,Sudan
ID_C20R2C0AYIT0,14.419128,33.528450,0.0,True,Sudan
...,...,...,...,...,...
ID_6LVHE89NN5VE,34.077906,70.697531,NaN,False,Afghanistan
ID_M51GDSUBKS8Q,32.202177,48.344005,NaN,False,Iran
ID_469MTLRKJC64,32.340069,48.157425,NaN,False,Iran


In [26]:
# Load the optical data if available locally, otherwise download it from GEE
optical_df = []
tasks = []

for country in country_settings.values():
    # We load 1 year before and 1 year after the start and end date 
    start_date = (pd.to_datetime(country[START_DATE]) - pd.DateOffset(days=365)).strftime('%Y-%m-%d')
    end_date = (pd.to_datetime(country[END_DATE]) + pd.DateOffset(days=365)).strftime('%Y-%m-%d')

    filename = (f'{COLLECTION_NAME.replace("/", "_")}_'
        + f'{country[COUNTRY_NAME]}_'
        + f'{str(start_date)}_{str(end_date)}'
    )

    # 1. Check if the file exists locally
    try:
        optical_df.append(pd.read_csv(CACHE_FOLDER + filename + '.csv'))
        continue
    except FileNotFoundError:
        print(f'File for {country[COUNTRY_NAME]} does not exist. We will download it from GEE first.')

    # 2. If not, check if it exists in GEE
    try:
        stats_fcc = ee.FeatureCollection(BASENAME + filename)
        stats_dict = fc_to_dict(stats_fcc).getInfo()

        df_country = prepare_df(df[filter_by_country(df, country)], stats_dict)
        df_country.to_csv(CACHE_FOLDER + filename + '.csv', index=False)
        optical_df.append(df_country)
        continue
    except ee.EEException as e:
        print(f'File for {country[COUNTRY_NAME]} does not exist in GEE. We will start a task to download it.')

    # 3. If not, start a task to download it in GEE
    task = start_download_task(df, country, start_date, end_date, ALL_BANDS, filename)
    tasks.append([task, country[COUNTRY_NAME]])

# If there are tasks running, wait for them to finish
if len(tasks) > 0:
    print('Downloading from ee...')
    print('This may take a while (up to 40 minutes).')
    print('This requires a Google Earth Engine account and at least 30MB of available storage space.')
    print('Please note that this download step is only required once.')

    wait_for_tasks(tasks)

    # Once the tasks are finished, load the data from GEE and save it locally
    for country in countries_settings.values():
        start_date = (pd.to_datetime(country[START_DATE]) - pd.DateOffset(days=365)).strftime('%Y-%m-%d')
        end_date = (pd.to_datetime(country[END_DATE]) + pd.DateOffset(days=365)).strftime('%Y-%m-%d')

        filename = (f'{COLLECTION_NAME.replace("/", "_")}_'
            + f'{country[COUNTRY_NAME]}_'
            + f'{str(start_date)}_{str(end_date)}'
        )
        
        stats_fcc = ee.FeatureCollection(BASENAME + filename)
        stats_dict = fc_to_dict(stats_fcc).getInfo()

        df_country = prepare_df(df[filter_by_country(df, country)], stats_dict)
        df_country.to_csv(CACHE_FOLDER + filename + '.csv', index=False)
        optical_df.append(df_country)

# Concatenate all country dataframes into a single one
optical_df = pd.concat(optical_df)

optical_df = optical_df.set_index(ID)
display(optical_df)


File for Sudan does not exist. We will download it first.
File for Afghanistan does not exist. We will download it first.
File for Iran does not exist. We will download it first.


,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,Lat,Lon,NDVI,SCL,Timestamp
ID,,,,,,,,,,,,,,,
ID_01EJ4DR99B1E,3134,2481,985,1410,2058,2459,2630,2759,2680,3015,14.347802,33.251769,0.131279,5,2018-12-14 08:26:09.000
ID_01EJ4DR99B1E,3066,2345,1052,1594,2316,2570,2691,2783,2984,3060,14.347802,33.251769,0.126038,5,2018-12-19 08:26:06.000
ID_01EJ4DR99B1E,2868,2280,978,1424,2122,2391,2554,2662,2776,2834,14.347802,33.251769,0.133524,5,2018-12-24 08:26:11.000
ID_01EJ4DR99B1E,2960,2304,1030,1524,2242,2387,2518,2745,2884,2931,14.347802,33.251769,0.125244,5,2018-12-29 08:26:08.000
ID_01EJ4DR99B1E,2822,2248,903,1334,1948,2218,2377,2585,2520,2721,14.347802,33.251769,0.128021,5,2019-01-03 08:26:13.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_ZZR0ZCVIZU0E,2306,1505,550,845,794,1401,2787,3395,3688,3604,32.131390,48.485220,0.645694,4,2021-06-09 07:40:53.727
ID_ZZR0ZCVIZU0E,2386,1511,529,891,946,1424,2992,3392,3902,3659,32.131390,48.485220,0.609736,4,2021-06-14 07:40:53.094
ID_ZZR0ZCVIZU0E,2451,1710,805,1140,1230,1666,2773,3290,3382,3450,32.131390,48.485220,0.466609,4,2021-06-19 07:40:53.463


We now have : 
- `df` : contains train and test data from 'Train.csv' and 'Test.csv'
- `optical_df` : conatains train and test optical timeseries data from GEE Sentinel-2

In `optical_df` each row is a timestep and once grouped by same `ID` they form the timeseries for each data point.

At this point, each time serie might be of different length. 
In the next step we will preprocess them to have a fixed length and aligned timesteps.

## 2. Pre-processing

In [27]:
bands = [B2, B3, B4, B8, LON, LAT, NDVI, SCL]

# For more details on how the timeranges were chosen, 
# please refer to section 2. of GEO_AI_Cropland_extent_antoine_saget.ipynb
country_settings_optimal = {
    SUDAN: {
        COUNTRY_NAME: SUDAN,
        START_DATE: '2019-05-29',
        END_DATE: '2021-03-31',
        BOUNDS: [[14.1, 33.1], [14.6, 33.6]]
    },
    AFGHANISTAN: {
        COUNTRY_NAME: AFGHANISTAN,
        START_DATE: '2021-06-27',
        END_DATE: '2023-04-30',
        BOUNDS: [[34.0, 70.2], [34.4, 70.8]]
    },
    IRAN: {
        COUNTRY_NAME: IRAN,
        START_DATE: '2018-08-28',
        END_DATE: '2020-06-30',
        BOUNDS: [[32.0, 48.1], [32.5, 48.6]]
    }
}

In [28]:
optical_df_filtered = pd.DataFrame(columns=[TIMESTAMP] + bands)

# Filtering out optical data outside of the optimal timeranges
for country in country_settings_optimal.values():
    country_mask = filter_by_country(df, country)
    country_mask = country_mask.reindex(optical_df.index, fill_value=False)

    optical = optical_df[country_mask]
    optical = filter_by_dates(optical, country[START_DATE], country[END_DATE])
    optical = optical[[TIMESTAMP] + bands]

    optical_df_filtered = pd.concat([optical_df_filtered, optical])

In [29]:
# One hot encoding of the SCL int column into multiple boolean columns
for scl in optical_df_filtered[SCL].unique():
    SCL_COL = f'{SCL}_{scl}'
    optical_df_filtered[SCL_COL] = optical_df_filtered[SCL] == scl
    optical_df_filtered[SCL_COL] = optical_df_filtered[SCL_COL].astype('uint8')
    bands.append(SCL_COL)
bands.remove(SCL)
optical_df_filtered = optical_df_filtered.drop(columns=[SCL])
optical_df_filtered = optical_df_filtered.reset_index(names=ID)

In [30]:
# Now let's interpolate the optical data to have a fixed and aligned number of timesteps
dfs_preprocessed = {}
for country in country_settings_optimal.values():
    country_mask = df[COUNTRY] == country[COUNTRY_NAME]
    ids = df.loc[country_mask].index
    
    optical_filtered = optical_df_filtered.loc[optical_df_filtered[ID].isin(ids)]
    X, IDs = interpolate_ts(optical_filtered, bands, start_date=optical_df_filtered[TIMESTAMP].min(), end_date=optical_df_filtered[TIMESTAMP].max())
    X = X.reshape(X.shape[0], -1)
    X = pd.DataFrame(X, index=IDs)

    X_train = X.loc[X.index.isin(df.loc[df[IS_TRAIN] & country_mask].index)].sort_index()
    Y_train = df.loc[df[IS_TRAIN] & country_mask, TARGET].sort_index().astype('uint8')
    X_test = X.loc[X.index.isin(df.loc[~df[IS_TRAIN] & country_mask].index)].sort_index()

    dfs_preprocessed[country[COUNTRY_NAME]] = {
        'X_train': X_train,
        'Y_train': Y_train,
        'X_test': X_test,
    }
    
print('Train data for SUDAN :')
display(dfs_preprocessed[SUDAN]['X_train'])
print('Train labels for SUDAN :')
display(dfs_preprocessed[SUDAN]['Y_train'])
print('Test data for SUDAN :')
display(dfs_preprocessed[SUDAN]['X_test'])

Train data for SUDAN :


,0,1,2,3,4,5,6,7,8,9,...,5430,5431,5432,5433,5434,5435,5436,5437,5438,5439
ID_07HVB7TS229V,1012.0,1302.0,1820.0,2238.0,33.258957,14.395682,0.103006,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_0A8LSLG2H2XH,10784.0,9896.0,8076.0,8832.0,33.333427,14.298844,0.044713,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_0DAEEQV9I7ED,735.0,763.0,768.0,859.0,33.379780,14.194459,0.055931,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_0HW4CNZ5CNEU,7636.0,6724.0,6244.0,6488.0,33.420921,14.292825,0.019164,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_0IIXJN8964FD,11752.0,10576.0,10096.0,9944.0,33.539230,14.317169,-0.007585,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_ZKL6M2FBVJ5G,12800.0,11208.0,10272.0,10840.0,33.534737,14.316271,0.026904,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_ZT9YQHPNE205,2208.0,2062.0,2350.0,2194.0,33.163822,14.179817,-0.034331,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_ZXCH3LDBSOT8,1130.0,1342.0,1592.0,1948.0,33.364868,14.165354,0.100565,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_ZYX3ZCDDZKVD,859.0,881.0,932.0,1072.0,33.569592,14.305401,0.069860,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Train labels for SUDAN :


ID
ID_07HVB7TS229V    1
ID_0A8LSLG2H2XH    1
ID_0DAEEQV9I7ED    0
ID_0HW4CNZ5CNEU    1
ID_0IIXJN8964FD    0
                  ..
ID_ZKL6M2FBVJ5G    0
ID_ZT9YQHPNE205    0
ID_ZXCH3LDBSOT8    0
ID_ZYX3ZCDDZKVD    1
ID_ZZZOKHJYLLVR    0
Name: Target, Length: 500, dtype: uint8

Test data for SUDAN :


,0,1,2,3,4,5,6,7,8,9,...,5430,5431,5432,5433,5434,5435,5436,5437,5438,5439
ID_01EJ4DR99B1E,7656.0,6904.0,6396.0,6584.0,33.251770,14.347802,0.014484,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_01NLOAIAX2Q8,962.0,1148.0,1428.0,1626.0,33.515873,14.255725,0.064833,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_03SYJY1J13A9,5220.0,5160.0,4956.0,4668.0,33.218800,14.255365,-0.029925,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ID_0APFLNEWTNK5,11272.0,9816.0,8968.0,8792.0,33.438351,14.390472,-0.009910,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ID_0AR1ZWJYNJP2,694.0,747.0,811.0,794.0,33.562496,14.206317,-0.010592,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_ZW46ETTOLKPW,14384.0,13192.0,12440.0,12080.0,33.342857,14.330824,-0.014682,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_ZYBJQKUP2Y70,8200.0,7992.0,7792.0,7276.0,33.526745,14.223295,-0.034245,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_ZYGY6FFVUCXW,13064.0,12120.0,11600.0,11256.0,33.229580,14.254197,-0.015051,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ID_ZYJJ32XRHM1I,385.0,367.0,418.0,654.0,33.257698,14.566901,0.220149,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Now we have fixed length timeseries with aligned timesteps separated in three country subsets.
For each country subset we have :
- `X_train` : the flatten timeserie train data, each row is a timeserie
- `Y_train` : the label corresponding to each timeserie
- `X_test` : the flatten timeserie test data, each row is a timeserie

# 3. Training the model

Given the relatively minor preprocessing, we use a RandomForest as it will be robust to problematic timesteps such as cloudy ones.
Indeed, the RandomForest will be able to ignore them and only use features that are relevant.
For good results with other methods, further preprocessing such as cloud filtering might be necessary.

For better generalization, we limit the depth of the trees in the random forest.
Also, we train one random forest per country.

In [31]:
PREDS = []
IDS = []

for country in country_settings_optimal.values():
    X_train = dfs_preprocessed[country[COUNTRY_NAME]]['X_train']
    Y_train = dfs_preprocessed[country[COUNTRY_NAME]]['Y_train']
    X_test = dfs_preprocessed[country[COUNTRY_NAME]]['X_test']

    model = RandomForestClassifier(random_state = SEED, n_estimators=100, max_depth=10)
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)

    PREDS += list(predictions)
    IDS += list(X_test.index.values)

PREDS = pd.Series(index=IDS, dtype='uint8', name='Pred', data=PREDS)
PREDS

ID_01EJ4DR99B1E    1
ID_01NLOAIAX2Q8    1
ID_03SYJY1J13A9    1
ID_0APFLNEWTNK5    0
ID_0AR1ZWJYNJP2    1
                  ..
ID_ZWL6GYN6VATQ    0
ID_ZX8D5A6AUZYI    0
ID_ZYAA2O5C9M2S    0
ID_ZZOSGGAB4PUY    0
ID_ZZR0ZCVIZU0E    1
Name: Pred, Length: 1500, dtype: uint8

In [32]:
# Compare predictions to subsmission to make sure they are the same
original = pd.read_csv('../submissions/10_05_19h_56m_49s_ts_rf_big_one_per_country_optimal_ranges.csv', index_col='ID', usecols=['ID', TARGET])
original[TARGET] = original[TARGET].astype('uint8')

diff = original.loc[IDS, TARGET] - PREDS
diff = diff[diff != 0]
print(f'Number of predictions different from original submission : {len(diff)}')

Number of predictions different from original submission : 0


In [ ]:
# Save the predictions
# save_submission(PREDS, IDS, 'ts_rf_ensemble_one_per_country_optimal_ranges')